# id, job, qualification, preferred, skills 추천 시스템

#### skills의 데이터로 qualification, preferred에 있는 skills 데이터 확인 후, filtering, 그 후 가중치 조절, 

In [1]:
import csv
import re

## 필요 함수 - getSkillsSet(), getSkillsList(), ListToDict(lst), saveCSV(dict, title)

#### getSkillsSet() - skills 리스트 빈도 필터링 없이 다 확인

In [2]:
# skills 데이터 다 확인

def getSkillsSet():
    with open("../data/crawled_data_combined.csv", encoding="utf-8") as f:
        reader = csv.reader(f)

        # 태그 하나씩 쪼개서 넣을 리스트 (set으로 중복 제거 전)
        skillList = []

        # csv 파일 리더로, 한 row씩 읽기
        for row in reader:

            # row list의 10번째 element인 tags string을 ''안 단어들로 나눠서 list만들기
            skills = re.findall(r"'(.*?)'", row[10])     
            skills_upper = [word.upper() for word in skills]

            # 각 줄의 태그를 tagList에 넣는다.
            skillList += skills_upper
            # print(skills)

        skillsSet = set(skillList)
        
        return skillsSet

#### getSkillsList() - 중복제거 없이 skill List 확인 함수 - 추후 해당 skill 빈도 확인을 위해 필요

In [3]:
# skills 데이터 다 확인

def getSkillsList():
    
    with open("../data/crawled_data_combined.csv", encoding="utf-8") as f:
        reader = csv.reader(f)

        # 태그 하나씩 쪼개서 넣을 리스트 (set으로 중복 제거 전)
        skillList = []

        # csv 파일 리더로, 한 row씩 읽기
        for row in reader:

            # row list의 10번째 element인 tags string을 ''안 단어들로 나눠서 list만들기
            skills = re.findall(r"'(.*?)'", row[10])     
            skills_upper = [word.upper() for word in skills]

            # 각 줄의 태그를 tagList에 넣는다.
            skillList += skills_upper
        
        return skillList

In [4]:
# skillsSet = getSkillsSet()
# skillsList = list(skillsSet)
# # skillsList

####  ListToDict(lst) - skills List로 딕셔너리 만들기

In [5]:
def ListToDict(lst):
    toDict = dict()

    for item in lst:
        toDict[item.upper()] = 0
    
    return toDict
    


#### saveCSV(dict, title) - dictionary key, value를 csv 파일로 저장하는 함수

In [6]:
import csv

def saveCSV(dict, title):
    # Specify the CSV file path

    # Open the CSV file in write mode
    with open(title, "w", newline="") as file:
        writer = csv.writer(file)

        # Write the header row with column names
        writer.writerow(["Key", "Value"])

        # Write each key-value pair as a row in the CSV file
        for key, value in dict.items():
            writer.writerow([key, value])

    print("Dictionary keys and values saved to CSV file successfully.")

#### addcount(set,lst) - 주어진 set에 있는 단어를 list에서 찾아서 해당 카운트 만큼 value값을 증가시킨 후 set return

In [7]:
def addcount(dict,lst):
    cnt = 1
    for i in dict:
        # print(i, lst.count(i))
        dict[i] = dict[i] + lst.count(i)
        cnt += 1
    # skills_filter = skills_filter.remove('백엔드 개발', 'REST')
    # print(cnt)
    return dict

## string값을 단어로 쪼개고, 필요없는 데이터 삭제 후, skill dict에 있는 단어 개수 확인 - qualification, preffered

#### 필요 라이브러리 import

In [8]:
comSet = getSkillsSet()
comList = list(comSet)
comDict = ListToDict(comList)

In [6]:
!pip install konlpy

#### qualification

In [28]:
from konlpy.tag import Komoran
import re
from tqdm import tqdm
import pandas as pd
import csv

def qualification():
    skillDict = comDict.copy()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            data = row[7].upper()

            # remove symbols
            data = re.sub(r'[^\w\s]', '', data)
            print(data)

            # Initialize Komoran
            komoran = Komoran()

            # Define a list of words to be preserved as one unit
            special_words = comList

            # Create a user dictionary file
            # user_dictionary = '\n'.join(special_words)

            

            # Your sentence
            # sentence = 'I am studying computer science and data science.'

            # Split the sentence into words using Komoran
            for word in special_words:
                data = data.replace(word, word.replace(' ', '_'))

            words = komoran.morphs(data)
            words = [word.replace('_', ' ') if '_' in word else word for word in words]

            # Print the words
            print(words)

            # komoran = Komoran()

            # words_together = comList
            # words_together_string = "_".join(words_together)

            # data_with_words_together = data + " " + words_together_string
            # # split text into morphemes
            # morphemes = komoran.morphs(data_with_words_together)
            # print(morphemes)
            # words = komoran.morphs(data)

            # # Filter out auxiliary words
            # words = [word for word in morphemes if not komoran.pos(word)[0][1].startswith('EF')]

    
            for skill in skillDict.keys():
                
                if skill in words:
                    skillDict[skill] = skillDict[skill] + 1

            if count % 500 == 0:
                print(skillDict)
            count += 1
    saveCSV(skillDict, "qual_dict_20230703.csv")
    return skillDict
            

# qualifications = 7, preferred = 8, skills = 10

In [29]:
qualification() 

  0%|          | 0/5451 [00:00<?, ?it/s]

 1년이상 JAVA 경력  SPRING BOOT 경험자


  0%|          | 1/5451 [00:01<1:44:21,  1.15s/it]

['1년', '이상', 'JAVA', '경력', 'SPRING', ' ', 'BOOT', '경험자']
 DBA 경력 5년 이상이신 분  대용량 DBMS 운영 경험하신 분  PUBLIC CLOUD AWS 환경에 대한 기본적인 이해 및 경험이 있는 분  쿼리 튜닝 분석 가능하신 분  MONGODB 운영 트러블슈팅 경험하신 분  인프라 환경의 자동화를 추구하며 동료와 적극적으로 소통할 수 있는 분  더 나은 기술을 익히고 적용하는데 거리낌 없는 분  문서화 및 메뉴얼 작성에 거부감이 없는 분


  0%|          | 2/5451 [00:03<3:04:58,  2.04s/it]

...

['클라우드', '및', '서비스', '구축', '및', '운영', '경험', '6년', '이상', 'PUBLIC', 'CLOUD', 'AZURE', 'NAVER', 'CLOUD', 'PLATFORM', 'ETC', 'PROVIDER', '인프라', '아키', '택', '처', '구성', '경험', '도', '크', '어', '컨테이너', '에', '대하', 'ㄴ', '이해', 'AZURE', 'CLOUD', '관련', '기술', '보유자', 'VMSTATIC', 'WEBAPP', 'SERVICEAPP', 'SERVICEBLOB', 'STORAGE', 'ETC', 'MSA', '전환', '및', '관련', '경험', 'UBUNTUCENTOSLINUX', 'OS', '운영', '경험', 'KUBERNETES', 'ADMINISTRATION', 'TECHNIQUE']
 C 사용 가능  영상 기하학에 대한 이해  DIRECTX 파이프라인에 대한 이해  객체지향 프로그래밍OOP에 대한 이해


  1%|          | 32/5451 [00:58<3:00:57,  2.00s/it]

['C', '사용', '가능', '영상', '기하학', '에', '대하', 'ㄴ', '이해', 'DIRECTX', '파이프라인', '에', '대하', 'ㄴ', '이해', '객체', '지향', '프로그래밍', 'OOP', '에', '대하', 'ㄴ', '이해']
ㆍ관련 경력 3년이상  ㆍ실내 NAVIGATION 분야 상용화 제품 개발 경력자저널 논문도 가능  ㆍGLOBAL PATH ALGORITHM 개발 경력자A 등등  ㆍLOCAL PATH PLANNING 개발 경력자DWA RRT 등등  ㆍROS C PYTHON 개발 능통  ㆍ탁월한 의사 소통 및 대인 관계 기술을 바탕으로 팀 중심 환경에서 작업할 수 있는 능력  ㆍ비즈니스 회화 능력  ㆍGIT 경험자


  1%|          | 32/5451 [01:00<2:51:39,  1.90s/it]


KeyboardInterrupt: 

In [34]:
import re
# skillDict = comDict.copy()

phrases = comList

# Your sentence
sentence = '1년이상 JAVA 경력  SPRING BOOT 경험자'

pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

# Find all occurrences of the phrases in the sentence
matches = re.findall(pattern, sentence)

# Replace the phrases with a unique placeholder
for match in matches:
    sentence = re.sub(r'\b' + re.escape(match) + r'\b', match.replace(" ", "_"), sentence)

# Split the sentence into words
words = sentence.split()

# Restore the original phrases
words = [word.replace("_", " ") if "_" in word else word for word in words]

# Print the words
print(words)



['1년이상', 'JAVA', '경력', 'SPRING BOOT', '경험자']


In [37]:
from konlpy.tag import Komoran
import re
from tqdm import tqdm
import pandas as pd
import csv

def qualification():
    skillDict = comDict.copy()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            data = row[8].upper()
            phrases = comList
            
            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # Find all occurrences of the phrases in the sentence
            matches = re.findall(pattern, sentence)

            # Replace the phrases with a unique placeholder
            for match in matches:
                sentence = re.sub(r'\b' + re.escape(match) + r'\b', match.replace(" ", "_"), sentence)

            # Split the sentence into words
            words = sentence.split()

            # Restore the original phrases
            words = [word.replace("_", " ") if "_" in word else word for word in words]

            # Print the words
            print(words)

            for skill in skillDict.keys():
                
                if skill in words:
                    skillDict[skill] = skillDict[skill] + 1

            if count % 500 == 0:
                print(skillDict)
            count += 1
    saveCSV(skillDict, "pref_dict_20230703.csv")
    return skillDict
            

# qualifications = 7, preferred = 8, skills = 10

In [38]:
qualification()

  5%|▍         | 256/5451 [00:00<00:04, 1289.97it/s]

['GIT', '형상관리', '사용', '경험', '협업', '커뮤니케이션이', '원활하신', '분']
['클라우드', '환경에서', 'ISMS', '심사', '경험이', '있는', '분', '모델링', '설계', '지원', '검토', '가능하신', '분', 'SHELL', 'JAVASCRIPT', 'PYTHON', '등', '스크립트', '언어', '사용', '가능하신', '분', '업무', '자동화', '스크립트', '작성', '가능하신', '분']
['이런', '경험을', '우대합니다', 'REDIS', 'RABBITMQ', '등', '오픈', '소스', '플랫폼을', '이용한', '서비스', '개발운영', '경험', '비동기', 'EVENTDRIVEN', 'APPLICATION', '개발', '경험', '대용량', '시스템', '개발', '및', '운영', '경험', 'TERRAFORM', 'IACINFRASTRUCTURE', 'AS', 'CODE', '경험']
['컴퓨터시스템공학', '전기전자공학', '개발', '경험이', '있으신', '분']
['GOLANG', '개발', '경험자', '가상화DOCKER', 'KUBERNETES', '환경', '설계구축운영', '경험이', '있는', '분', '버전관리GIT', 'GITHUB', 'GITLAB에', '익숙하신', '분', 'GRAPHQL에', '대한', '지식이', '있는', '분', 'ISSUE', '관리JIRA', 'GITLAB에', '익숙하신', '분', 'OPEN', 'API', 'SPECIFICATION에', '대한', '지식이', '있는', '분', 'CODE', 'REVIEW에', '대한', '거부감이', '없는', '분', 'TEST', 'CODE', '작성', '경험자', '원활한', '커뮤니케이션', '능력이', '있는', '분']
['가상자산', '거래소', '또는', '금융', '관련', '프로젝트', '경험을', '보유하신', '분', '자산', '보안', '및', '관리에',

  7%|▋         | 386/5451 [00:00<00:04, 1215.75it/s]

['TCPIP', '에', '대한', '지식이', '풍부하신', '분', 'WINDOW', '환경에서', '웹', '어플리케이션', '개발', '경험이', '있으신', '분', '국제', '표준을', '바탕으로', 'PRODUCT', '를', '개발', '해', '보신', '경험이', '있으신', '분']
['해당경험우대', '데이터베이스', '설계구축운영', '경험', '서버', '운영체제클라우드', '포함', '설계구축운영', '경험', 'JAVA', '개발', 'SPRING FRAMEWORK', '기반의', '애플리케이션', '개발', '결제서비스가', '포함된', '플랫폼', '개발', '경험', 'REACT를', '이용한', 'FRONTEND', '개발', '경험', '클라우드', '서비스AWS', 'GCP', '등', '운영', '경험', '빌드테스트배포', '자동화', '경험', 'MICROSERVICES', '아키텍처', '기반의', '시스템', '개발', '경험', '해당역량우대', 'PM', '역할', '수행', '및', '유지보수', '운영', '경험', '품질', '좋은', '시스템과', '코드를', '만들기', '위해', '항상', '노력하는', '분', '새로운', '기술을', '배우며', '동료와', '함께', '성장하기', '원하는', '분']
['ERP', '구축', '프로젝트', '경험자', '사내', '컨설팅', '및', '컨설팅', '유경험자', '프로젝트', '관련', '자격증', '소유자', 'PMP']
['우대', '하나의', '제품에서', '클라이언트', '측과', '서버', '측', '모두에서', '작업한', '경험', '우대', 'SWIFTUI', '경험', '우대', 'PYTORCH', '경험']
['AWS를', '활용한', '개발', '및', '운영', '경험', '도메인', '주도', '설계DDD', '지식', '및', '경험']
[]
['ㆍ', 'RESTFUL API', 'GRAPHQL', '경험이', '있

 14%|█▍        | 766/5451 [00:00<00:03, 1224.08it/s]

...

['AWS', 'DOCKER', 'HADOOP', '사용', '경험', '우대', '빅데이터', '관련', '기본지식']
['ATLASSIAN', '제품', 'JIRA', 'CONFLUENCE', 'BAMBOO', '등에', '대한', '구축', '및', '지원', '경험', 'GIT', 'JENKINS등', 'CICD', '사용', '경험', '아틀라시안', '솔루션', '연동', '개발', '경험']
['TYPESCRIPT', '개발', '및', '운영', '경험이', '있으신', '분', '페어', '코딩', '및', '코드', '리뷰', '경험이', '있으신', '분', '단위', '테스트', '통합', '테스트', '자동화', '및', '배포', '자동화', '경험이', '있으신', '분', '웹사이트', '성능 측정', '및', '최적화', '경험이', '있으신', '분', '웹사이트', '보안에', '대한', '이해', 'JAVASCRIPT', '관련', '오픈', '소스', '기여', '경험이', '있으신', '분', '백앤드', '서비스', '개발', '경험', '및', 'DATABASE에', '대한', '이해', 'RDBMS', 'NOSQL']
['그', '외', '이런', '것이', '있으면', '더', '좋아요', 'PYTHON', 'JAVASCRIPT', 'JAVA', 'C', '등과', '같은', '개발', '언어', '사용', '경험', 'JENKINS등의', 'CI', 'TOOL를', '사용한', '배포', '시스템 구축', '경험', 'GIT', '등의', '분산', '버전', '관리', '시스템에', '대한', '이해', 'KIBANA', 'DATADOGS', 'GRAFANA', 'NEWRELIC', '등과', '같은', '모니터링', '툴', '사용', '경험', '성능', '테스트', '수행', '포트폴리오', '관련', '학과', '전공', '기타', '사항', '고용형태', '정규직', '수습기간', '3개월']
['NO

100%|██████████| 5451/5451 [00:04<00:00, 1237.74it/s]

['패키지', '소프트웨어 개발', '경험', '상용', '소프트웨어 개발', '경험', 'SOFTWARE', 'ARCHITECTURE', '설계', '경험', 'APPLICATION', 'FRAMEWORK', '설계', '경험', 'CICD', '환경', '구축', '경험', '사용성', '평가를', '위한', '정보를', '수집', '환경', '구축', '경험']
['C', '또는', 'C', '개발', '경험', 'RESTFULAPI', '개발', '경험']
['우대조건', 'ㆍ스토어', '출시', '및', '관리', '유지보수', '경험이', '있으신', '분', 'ㆍ오픈', '소스에', '기여한', '적이', '있는', '분', 'ㆍIOS', 'ANDROIDSWIFT', 'KOTLIN', '네이티브', '개발', '경험이', '있는', '분', 'ㆍMVP', 'MVVM', '등의', '디자인', '패턴', '이해와', '개발', '적용', '경험이', '있으신', '분', 'ㆍCLEAN', 'ARCHITECTURE', 'CLEAN', 'CODE', '구현에', '깊이', '고민하고', '개선해', '보신', '분', 'ㆍ', 'BLE관련', '개발을', '해', '보신', '분', 'ㆍRESTFUL', 'API', '사용해', '보신', '분', 'ㆍGRADLE에', '대한', '이해와', '경험이', '있으신', '분', 'ㆍ의료', '및', '헬스케어', '디바이스를', '경험', '해보신', '분', 'ㆍJIRA', 'CONFLUENCE', 'GIT', '등', '협업툴', '사용', '경험이', '있으신', '분', 'ㆍ다양한', '분야의', '사람과의', '협업', '능력이', '원활하신', '분', 'ㆍ논리적이고', '체계적인', '문제해결', '능력', '및', '커뮤니케이션', '능력이', '있으신', '분']
['SAP', '관련', '자격증', '및', '교육', '경험', '보유', 'SAP', 'S4HANA시스템', '마이그레이션

Dictionary keys and values saved to CSV file successfully.


{'윈도우 프로그래밍': 2,
 'CTI': 3,
 'RASPBERRY PI': 3,
 'POS': 21,
 'GCP': 158,
 '모바일 기술': 0,
 '3G': 0,
 '회로 설계': 15,
 'IOS 디자인': 0,
 'OLAP': 6,
 '전력전자학': 0,
 'FLASK': 23,
 '클라우드 스토리지': 0,
 'ANGULARJS': 8,
 '모바일 게임 개발': 0,
 'OPENCL': 6,
 'SMTP': 0,
 'RESTFUL WEBSERVICES': 0,
 'AXURE': 3,
 'HYPERLEDGER': 5,
 'XEN': 0,
 'B2B': 44,
 'QT': 15,
 '제품 개발': 29,
 'ITIL': 6,
 '전자정부프레임워크': 4,
 'MICROSOFT SERVER': 0,
 'RUBY ON RAILS': 9,
 'TABLEAU': 27,
 '기술 교육': 0,
 'FIGMA': 58,
 'ADC': 1,
 'COMMUNICATION': 19,
 '신호 처리': 3,
 'REDIS': 148,
 'LINQ': 1,
 'ES6': 24,
 'RTL 검증': 0,
 'POP 디스플레이': 0,
 'NODE.JS': 0,
 'LARAVEL': 14,
 'JIRA': 198,
 'FASTAPI': 17,
 'IOS 개발': 14,
 'PCB 디자인': 0,
 'SSRS': 0,
 'POSTGRESQL': 56,
 'C': 135,
 'ECLIPSE': 1,
 '설계': 515,
 'GRAPHQL': 61,
 'PMP': 7,
 'NUMPY': 7,
 'EDA': 6,
 '서비스 프로세스': 0,
 '학술 출판': 0,
 '솔루션 아키텍처': 2,
 'SIMULINK': 4,
 'XCODE': 7,
 'EMC 규정 준수': 0,
 'XML': 3,
 'OTN': 0,
 'PRODUCT MANAGEMENT ': 0,
 'CAN': 7,
 'DJANGO': 39,
 'COCOA TOUCH': 0,
 'CSS': 64,
 'GCPS': 0

#### preferred

In [ ]:
from konlpy.tag import Komoran
import re
from tqdm import tqdm
import pandas as pd
import csv

def preffered():
    skillDict = comDict.copy()

    with open("../data/crawled_data_combined.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        for row in tqdm(data[1:]):
            data = row[8].upper()

            # remove symbols
            data = re.sub(r'[^\w\s]', '', data)
            komoran = Komoran()

            # split text into morphemes
            morphemes = komoran.morphs(data)
            # words = komoran.morphs(data)

            # Filter out auxiliary words
            words = [word for word in morphemes if not komoran.pos(word)[0][1].startswith('EF')]

            for skill in skillDict.keys():
                
                if skill in words:
                    skillDict[skill] = skillDict[skill] + 1

            if count % 500 == 0:
                print(skillDict)
            count += 1

    saveCSV(skillDict, "pref_dict.csv")
    return skillDict
            

# qualifications = 7, preferred = 8, skills = 10

#### Skills

In [46]:
import re

def skills():
    totalList = getSkillsList()
    skillDict = comDict.copy()
    
    skills_count = addcount(skillDict, totalList)
    saveCSV(skills_count, "skill_dict_20230703.csv")
    return skills_count

In [47]:
skills()

Dictionary keys and values saved to CSV file successfully.


{'윈도우 프로그래밍': 2,
 'CTI': 4,
 'RASPBERRY PI': 5,
 'POS': 1,
 'GCP': 117,
 '모바일 기술': 4,
 '3G': 4,
 '회로 설계': 19,
 'IOS 디자인': 2,
 'OLAP': 1,
 '전력전자학': 2,
 'FLASK': 60,
 '클라우드 스토리지': 1,
 'ANGULARJS': 71,
 '모바일 게임 개발': 1,
 'OPENCL': 4,
 'SMTP': 17,
 'RESTFUL WEBSERVICES': 36,
 'AXURE': 7,
 'HYPERLEDGER': 5,
 'XEN': 14,
 'B2B': 5,
 'QT': 15,
 '제품 개발': 1,
 'ITIL': 14,
 '전자정부프레임워크': 4,
 'MICROSOFT SERVER': 2,
 'RUBY ON RAILS': 25,
 'TABLEAU': 62,
 '기술 교육': 4,
 'FIGMA': 112,
 'ADC': 3,
 'COMMUNICATION': 4,
 '신호 처리': 3,
 'REDIS': 145,
 'LINQ': 4,
 'ES6': 26,
 'RTL 검증': 8,
 'POP 디스플레이': 5,
 'NODE.JS': 456,
 'LARAVEL': 42,
 'JIRA': 247,
 'FASTAPI': 58,
 'IOS 개발': 52,
 'PCB 디자인': 19,
 'SSRS': 1,
 'POSTGRESQL': 214,
 'C': 276,
 'ECLIPSE': 2,
 '설계': 6,
 'GRAPHQL': 96,
 'PMP': 19,
 'NUMPY': 28,
 'EDA': 14,
 '서비스 프로세스': 1,
 '학술 출판': 2,
 '솔루션 아키텍처': 4,
 'SIMULINK': 17,
 'XCODE': 19,
 'EMC 규정 준수': 7,
 'XML': 2,
 'OTN': 2,
 'PRODUCT MANAGEMENT ': 53,
 'CAN': 11,
 'DJANGO': 146,
 'COCOA TOUCH': 1,
 'CSS': 2

## qual_dict, pref_dict, skills_dict 3개의 csv파일을 통합하기

#### qual_dict keys, values확인

In [25]:
import csv

with open("qual_dict.csv", 'r', encoding="949") as f:
    reader = csv.reader(f)
    
    qual_keys = []
    qual_values = []

    for row in reader:
        qual_keys.append(row[0])
        qual_values.append(row[1])


#### pref_dict keys, values확인

In [24]:
import csv

with open("pref_dict.csv", 'r', encoding="949") as f:
    reader = csv.reader(f)
    
    pref_keys = []
    pref_values = []

    for row in reader:
        pref_keys.append(row[0])
        pref_values.append(row[1])


#### skill_dict keys, values확인

In [45]:
import csv

with open("skill_dict.csv", 'r', encoding="949") as f:
    reader = csv.reader(f)
    
    skill_keys = []
    skill_values = []

    for row in reader:
        skill_keys.append(row[0])
        skill_values.append(row[1])


#### Keys 중에 순서가 다른 element가 있는지 확인

In [29]:
for i in range(len(skill_keys)):
    cnt = 0
    if skill_keys[i] != pref_keys[i]:
        cnt += 1
    elif skill_keys[i] != qual_keys[i]:
        cnt += 1
    elif pref_keys[i] != qual_keys[i]:
        cnt +=1 
print(cnt)



0


#### 3개 데이터 한 csv에 합치기

In [32]:
import pandas as pd

# Create the DataFrame
df = pd.DataFrame({'keys': skill_keys, 'qualification': qual_values, 'preffered': pref_values, 'skill': skill_values})

# Save the DataFrame as a CSV file with column names
df.to_csv('qual_pref_skill_count.csv', index=False, encoding="utf-8")
